# Azure AI Agent Service with Bing Grounding

이 노트북은 Azure AI Agent Service와 Bing Grounding Tool을 활용하여, 사용자 질문에 대해 Bing 검색 결과를 기반으로 답변을 생성하는 예제를 다룹니다.

## 주요 내용

- Azure AI Project Client와 Bing Grounding Tool을 초기화합니다.
- Bing 연결 정보를 불러와서 Grounding Tool을 구성합니다.
- 사용자 프롬프트(질문)를 입력하면, Bing 검색을 활용한 답변을 생성하는 Agent를 동적으로 생성하고 실행합니다.
- 각 질문에 대해 에이전트의 답변과 참고한 URL 정보를 출력합니다.

이 과정을 통해, 실제 Bing 검색 결과를 기반으로 한 최신 정보 답변을 Azure AI Agent를 통해 자동화하는 방법을 실습할 수 있습니다.





In [2]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import MessageRole, BingGroundingTool
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import AzureOpenAI
import json
load_dotenv(override=True, dotenv_path="../.env")

credential = DefaultAzureCredential()


In [2]:
from azure.ai.projects.models import ThreadMessage

project_client = AIProjectClient.from_connection_string(
    credential=credential,
    conn_str=os.environ["AZURE_AI_PROJECT_CONNECTION_STRING"],
)

bing_connection = project_client.connections.get(
    connection_name=os.environ["BING_CONNECTION_NAME"],
    include_credentials=True
)
conn_id = bing_connection.id

print(conn_id)

bing = BingGroundingTool(connection_id=conn_id)

def populate_answer_agent_with_bing_grounding_tool(prompt: str, project_client: AIProjectClient, bing) -> ThreadMessage:
    agent = project_client.agents.create_agent(
        model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
        name="my-assistant",
        instructions="너는 삼성전자 제품 관련 정보를 제공하는 챗봇이야. 답변은 마크다운으로 작성해줘.",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"},
        top_p=0.9,
    )

    thread = project_client.agents.create_thread()

    project_client.agents.create_message(
        thread_id=thread.id,
        role=MessageRole.USER,
        content=prompt,
        
    )

    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id, max_completion_tokens=1500)

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    project_client.agents.delete_agent(agent.id)
    response_message = project_client.agents.list_messages(thread_id=thread.id).get_last_message_by_role(
        MessageRole.AGENT
    )

    return  response_message



def display_markdown_content(content: str):
    display(Markdown(content))

/subscriptions/ef750ece-3939-4a4b-ade4-cae9eabcf312/resourceGroups/rg-writer/providers/Microsoft.MachineLearningServices/workspaces/ai-project-q4u67epqs2xo4/connections/agentbingsearch


In [ ]:
from azure.ai.projects.models import ThreadMessage

project_client = AIProjectClient.from_connection_string(
    credential=credential,
    conn_str=os.environ["AZURE_AI_PROJECT_CONNECTION_STRING"],
)

bing_connection = project_client.connections.get(
    connection_name=os.environ["BING_CONNECTION_NAME"],
    include_credentials=True
)
conn_id = bing_connection.id

print(conn_id)

bing = BingGroundingTool(connection_id=conn_id)

agent_id = "asst_gWJv98cVPhvGWCdzXTIYahe0"

def populate_answer_agent_with_bing_grounding_tool(prompt: str, project_client: AIProjectClient, bing) -> ThreadMessage:
    agent = project_client.agents.create_agent().get_agent(agent_id)

    thread = project_client.agents.create_thread()

    project_client.agents.create_message(
        thread_id=thread.id,
        role=MessageRole.USER,
        content=prompt,
        
    )

    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id, max_completion_tokens=1500)

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # project_client.agents.delete_agent(agent.id)
    response_message = project_client.agents.list_messages(thread_id=thread.id).get_last_message_by_role(
        MessageRole.AGENT
    )

    return  response_message



def display_markdown_content(content: str):
    display(Markdown(content))

/subscriptions/ef750ece-3939-4a4b-ade4-cae9eabcf312/resourceGroups/rg-writer/providers/Microsoft.MachineLearningServices/workspaces/ai-project-q4u67epqs2xo4/connections/agentbingsearch


In [8]:
QUERY_REWRITE_PROMPT = """
            <<지시문>>
            너는 LLM 질의 최적화 전문가야. 사용자가 입력한 질문을 두 가지 목적에 맞게 재작성해.
            
            1. LLM Query용 Rewrite:
            - 사용자의 질문을 LLM이 더 잘 이해하고 답변할 수 있도록, 맥락과 의도를 명확히 드러내는 자연스러운 문장으로 재작성해.
            - 필요한 경우 추가 설명이나 세부 조건을 포함해서 질문의 목적이 분명히 드러나도록 만들어.
            - LLM이 답변에 집중할 수 있도록 핵심 단어를 반복 사용해.

            <<예시>>
            * 질문: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘            
            * LLM 답변용 재작성: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.

            <<질문>>
            {user_query}

            <<출력포맷>>
            반드시 아래와 같이 json 형식으로 출력해.
            {"llm_query": "LLM 답변용 재작성"}
        """     
  
def rewrite_query_for_search_and_llm(query, client: AzureOpenAI):
        response = client.chat.completions.create(
            model=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
            messages=[
                {"role": "system", "content": QUERY_REWRITE_PROMPT},
                {"role": "user", "content": query}
            ],
            temperature=0.8,
            max_tokens=300,
            response_format= {"type": "json_object"},
        )
        
        return json.loads(response.choices[0].message.content.strip())

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-08-01-preview"
)

inputs = [
    "삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘",
    "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
    "삼성전자 25년 제품이 작년 대비 좋아진것은",
    "삼성전자 JBL과 하만카돈 차이점이 뭐야",
    "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
]

for input in inputs:
    print(f"User input: {input}")
    rewritten_query = rewrite_query_for_search_and_llm(input, client)
    print(f"rewritten query: {rewritten_query}")
    response_message = populate_answer_agent_with_bing_grounding_tool(rewritten_query['llm_query'], project_client, bing)
    if response_message:
        for text_message in response_message.text_messages:
            display_markdown_content(text_message.text.value)
        # for annotation in response_message.url_citation_annotations:
        #     print(f"URL Citation: [{annotation.url_citation.title}]({annotation.url_citation.url})")


User input: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘


삼성전자 인덕션 중 3구 이상 또는 다양한 화구 수를 갖춘 모델 중에서 추천드릴 수 있는 제품과 그 주요 기능, 장점은 다음과 같습니다.

### 1. 삼성 인덕션 3구 모델 (예: NZ63T9777GK)
- **주요 기능:**
  - 3개의 화구로 동시에 조리 가능
  - 스마트 화력 조절 기능으로 정교한 화력 조절 가능
  - Wi-Fi 연결 지원으로 스마트폰 앱을 통한 원격 조작 가능
  - 안전 잠금 기능 및 자동 전원 차단 기능 탑재
- **장점:**
  - 다중 요리 동시에 효율적 조리 가능
  - 스마트 기능으로 사용자 편의성 증가
  - 안전장치 탑재로 안전하게 사용 가능

### 2. 삼성 인덕션 4구 모델 (예: NZ64T8757GK)
- **주요 기능:**
  - 4개의 화구로 대가족이나 대용량 조리에 적합
  - 인공지능 쿡탑 기능 탑재로 요리 최적화
  - 전용 그릴 기능 및 다양한 조리 모드 제공
  - 자동 온도 유지 및 화구별 독립 제어 가능
- **장점:**
  - 더 많은 요리를 동시에 조리할 수 있어 시간 절약
  - 다양한 조리 모드로 요리의 완성도 향상
  - 편리한 제어 인터페이스로 사용자 경험 향상

### 3. 삼성 인덕션 5구 이상 고급형 모델 (예: NZ98T9777GK)
- **주요 기능:**
  - 5구 이상의 넉넉한 화구 수로 프로급 요리 환경 제공
  - 터치 방식 스마트 컨트롤과 사용자 맞춤형 인터페이스
  - 강력한 화력과 균일한 열 분포 기술
  - 자동 냄비 인식 및 화구간 전환 기능
- **장점:**
  - 전문 요리사급 조리 환경 구성 가능
  - 다채로운 요리 스타일을 지원하는 높은 자유도
  - 고급 안전 기능과 인체 공학적 디자인 적용

---

위 모델들은 삼성 공식 홈페이지나 대리점을 통해 상세 스펙과 가격을 확인하실 수 있습니다. 필요에 따라 사용 공간, 요리 스타일, 인원 수 등을 고려해 적합한 화구 수의 인덕션을 선택하시면 좋습니다. 추가로 궁금한 점 있으면 알려주세요!

User input: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘


부모님께 선물하기 좋은 삼성전자 TV 모델을 추천드리기 위해, 일반적으로 고려할 사항과 함께 추천 모델을 알려드릴게요.

---

## 1. 주요 고려 사항

### 예산
- **중저가형** : 50~70만 원대 (가성비 좋은 기본형 모델)
- **중급형** : 80~150만 원대 (적절한 기능과 화질)
- **고급형** : 150만 원 이상 (최신 기술과 프리미엄 디자인/화질)

### 화면 크기
- 부모님 거실 환경에 따라 43인치~65인치가 일반적입니다.
- 거실이 넓으면 55인치 이상을 추천드립니다.

### 화질 및 기능
- **QLED**: 밝고 선명한 색감과 뛰어난 화질 제공
- **4K 해상도**: 선명한 화질, 고해상도 영상 감상 가능
- **스마트 TV 기능**: 유튜브, 넷플릭스 등 스트리밍 앱 지원
- **음향 지원**: 만족스러운 사운드 출력 (별도 사운드바 연동 가능)
- **사용 편의성**: 심플한 리모컨 및 간편한 UI

---

## 2. 추천 모델

### ① 삼성 QN90B QLED 4K Neo QLED TV
- 예산: 고급형
- 화면 크기: 55인치, 65인치 등
- 장점: 삼성의 최신 네오 QLED 기술 적용, 뛰어난 명암비와 색 재현, 밝은 환경에서도 탁월한 화질, 스마트 기능과 매끄러운 OS
- 적합: 화질과 성능을 중시하는 분께 추천

### ② 삼성 AU8000 Crystal UHD TV
- 예산: 중저가형 ~ 중급형
- 화면 크기: 43인치, 50인치, 55인치 등 다양
- 장점: 가성비 좋은 4K 화질, 기본적인 스마트 TV 기능, 무난한 음질과 심플한 디자인
- 적합: 부담 없는 가격에 무난한 TV를 원할 때 추천

### ③ 삼성 The Frame TV
- 예산: 중급형 ~ 고급형
- 화면 크기: 43인치, 50인치, 55인치
- 장점: 액자형 디자인 (예술 작품을 TV 꺼진 상태에서도 감상 가능), 높은 화질, 인테리어용 TV로 인기
- 적합: 인테리어를 중시하고 TV를 예술 작품처럼 활용하고 싶은 분께 추천

---

필요하시면 예산과 사용 환경에 맞춘 구체적인 모델과 구매처 정보도 도와드릴 수 있습니다!

User input: 삼성전자 25년 제품이 작년 대비 좋아진것은


...

User input: 삼성전자 JBL과 하만카돈 차이점이 뭐야


삼성전자에서 판매하는 JBL 브랜드와 하만카돈(Harman Kardon) 브랜드는 모두 하만(Harman)의 오디오 브랜드로, 각각의 특성과 시장 포지션에서 차이가 있습니다. 두 브랜드는 음질, 디자인, 가격, 제품 라인업 등 여러 면에서 차별화되어 있습니다.

---

## 1. 브랜드 개요

- **JBL**
  - 대중적이고 활동적인 이미지.
  - 주로 휴대용 스피커, 헤드폰, 사운드바 등 다양한 생활 음악기기 중심.
  - 강력하고 풍부한 저음 중심의 사운드가 특징.
  - 스포츠, 아웃도어, 파티용으로 많이 선호되는 음향 장비를 주로 판매.

- **하만카돈 (Harman Kardon)**
  - 프리미엄 오디오 브랜드로 고급스러운 디자인과 음질에 중점.
  - 주로 고품질 홈 오디오, 하이엔드 스피커, 사운드바 등을 중심으로 제품을 출시.
  - 중음과 고음의 섬세한 표현력과 밸런스 좋은 사운드가 특징.
  - 디자인 면에서 세련되고 미니멀한 스타일을 강조.

---

## 2. 음질

- **JBL**
  - 베이스가 강하고 파워풀한 음향을 선호하는 사용자에게 적합.
  - 음악 재생 시 강한 저음 효과로 신나는 분위기 연출에 좋음.
  - 야외 활동이나 파티용으로 최적화.

- **하만카돈**
  - 중저음, 중음, 고음의 밸런스가 뛰어나 보다 정교하고 자연스러운 음질 제공.
  - 하이파이(Hi-Fi)급 사운드 제작에 적합.
  - 클래식, 재즈, 보컬 음악 등 음악의 디테일한 부분까지 살리는 음향 특성.

---

## 3. 디자인

- **JBL**
  - 대체로 튼튼하고 스포티한 디자인.
  - 컬러풀하고 다양한 사이즈의 제품이 많아 선택의 폭이 넓음.
  - 휴대성과 내구성을 중시한 외관 구성.

- **하만카돈**
  - 고급스러운 메탈 및 우드 소재 사용, 미니멀하고 세련된 디자인.
  - 인테리어 소품처럼 공간에 어울리는 프리미엄 느낌 강조.
  - 부드러운 곡선과 깔끔한 라인이 특징.

---

## 4. 가격

- **JBL**
  - 대체로 합리적인 가격대.
  - 보급형부터 중급형까지 폭넓은 선택지가 있으며, 가격 대비 성능이 뛰어난 편.

- **하만카돈**
  - 프리미엄 가격대로 다소 높은 편.
  - 고음질과 디자인을 우선시하는 사용자를 위한 고가 제품군 위주.

---

## 5. 제품 라인업

- **JBL**
  - 포터블 블루투스 스피커(Flip, Charge, Pulse 시리즈 등)
  - 무선 이어폰 및 헤드폰
  - 자동차 오디오 및 홈 오디오 제품
  - 사운드바, 파티 스피커 등

- **하만카돈**
  - 프리미엄 블루투스 스피커(Onyx Studio, Aura 등)
  - 하이파이 스피커 시스템
  - 고급 사운드바 및 홈 씨어터 오디오
  - 차량용 프리미엄 사운드 시스템

---

## 요약

| 구분       | JBL                              | 하만카돈 (Harman Kardon)           |
|------------|---------------------------------|----------------------------------|
| 음질       | 강력하고 파워풀한 베이스 중심      | 정교하고 밸런스 좋은 하이파이 음질  |
| 디자인     | 스포티하고 튼튼한 컬러풀 디자인   | 고급스럽고 세련된 미니멀 디자인    |
| 가격       | 합리적, 보급형부터 중급형 다양     | 프리미엄, 고가 제품 중심            |
| 제품 라인업 | 휴대용 스피커, 헤드폰, 사운드바 등 다양 | 하이파이 스피커, 프리미엄 사운드바 등 |

---

필요하신 특정 제품 추천이나 모델별 상세 비교도 가능하니 언제든 말씀해 주세요!

User input: 갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요


갤럭시 버즈 이어버드 한쪽을 새로 구매했을 때, 새로 산 이어버드와 기존 이어버드를 함께 사용하는 방법은 다음과 같습니다. 한쪽 이어버드만 교체하는 경우, 새 이어버드를 기존 케이스와 기존 이어버드와 함께 정상 페어링하여 사용할 수 있도록 설정하는 절차와 주의사항을 안내드립니다.

---

## 1. 갤럭시 버즈 이어버드 한쪽 교체 시 페어링 절차

### 1) 기본 준비
- 기존 갤럭시 버즈(한쪽 이어버드 포함) 및 새로 구매한 이어버드를 준비합니다.
- 갤럭시 버즈 충전 케이스도 함께 준비합니다.
- 충전 케이스 배터리가 충분한지 확인합니다.

### 2) 새 이어버드 충전 케이스에 넣기
- 새로 산 한쪽 이어버드와 기존 이어버드를 모두 충전 케이스에 동시에 넣습니다.
- 이어버드가 제대로 충전케이스에 자리 잡히고 충전이 시작되는지 확인합니다.
- 두 이어버드 모두 케이스에 들어간 상태여야 페어링이 제대로 진행됩니다.

### 3) 충전 케이스 닫고 다시 열기
- 이어버드를 넣은 후 충전 케이스를 한번 닫은 뒤 다시 열어 신규 이어버드와 기존 이어버드가 함께 켜져서 페어링 준비가 되도록 합니다.

### 4) 이어버드 페어링 모드 진입
- 갤럭시 버즈 이어버드를 페어링 모드로 진입시키려면, 충전 케이스 뚜껑을 열고 이어버드를 케이스에서 꺼냅니다.
- 이어버드의 불빛이 깜박이는 것을 확인하여 페어링 모드를 활성화합니다.
- 스마트폰에서 Bluetooth 메뉴로 이동해 ‘Galaxy Buds’ 기기를 찾습니다.

### 5) 스마트폰에서 다시 페어링 시도
- 스마트폰 블루투스 설정에서 기존에 연결된 ‘Galaxy Buds’ 기기를 삭제(Remove/Forget)합니다.
- 이어버드 양쪽이 제대로 충전 케이스에 들어가 있었다면, 다시 한번 Bluetooth에서 ‘Galaxy Buds’를 검색하고 페어링합니다.
- 페어링 완료 시 양쪽 새 이어버드와 기존 이어버드가 함께 기능합니다.

---

## 2. 한쪽 이어버드만 사용하는 경우 주의사항

- 한쪽 이어버드만 단독으로 페어링하여 사용할 경우, 제품에 따라 음질이나 통화 품질이 저하될 수 있습니다.
- 갤럭시 버즈는 두 이어버드를 동시에 페어링할 때 최적의 사운드 밸런스를 제공합니다.
- 새 이어버드가 기존 이어버드와 정상적으로 페어링되지 않는 경우, 갤럭시 웨어러블 앱(Galaxy Wearable app)을 통해 펌웨어 업데이트 및 재설정을 시도해야 합니다.
- 이어버드 개별 초기화(재설정)를 원한다면, 갤럭시 웨어러블 앱 내 ‘이어버드 재설정’ 기능을 사용하면 됩니다.

---

## 3. 추가 팁

- 새 이어버드가 기존 버즈와 동일한 모델이고, 최신 펌웨어로 업데이트가 되어 있어야 호환 문제가 적습니다.
- 충전 케이스에 두 이어버드를 넣었을 때 LED 불빛 색과 깜박임 상태로 정상 인식되는지 반드시 확인하세요.
- 페어링 및 사용 중 문제가 발생하면, 이어버드와 케이스를 완전 초기화 후 다시 시도하는 게 좋습니다.

---

필요하다면 갤럭시 웨어러블 앱 설치 및 사용 방법도 안내드릴 수 있습니다. 더 궁금한 점 있으면 알려 주세요!

User input: 삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나
Run failed: {'code': 'server_error', 'message': 'Sorry, something went wrong.'}
